In [2]:
!pip install mediapipe opencv-python numpy

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

def calculate_distance(p1, p2):
    return np.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

def count_fingers(hand_landmarks, hand_label="Right"):
    """
      hand_landmarks - landmarks
      hand_label -  "Right" ot "Left" string, which dedicate right or left hand
    returns:
      count - sum of the fingers.
    """
    count = 0  
    # Thumb: X-axis check depends on handedness
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip  = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    if hand_label == "Right":
        if thumb_tip.x < thumb_ip.x:
            count += 1
    else:  # для левой руки
        if thumb_tip.x > thumb_ip.x:
            count += 1
    # Other fingers: Y-axis check
    if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y:
        count += 1
    if hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y:
        count += 1
    if hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y:
        count += 1
    if hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y:
        count += 1

    return count


# Gesture recognition function for operators, confirmation and calculation
def get_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # Operators:
    # Addition: Thumbs Up👍
    if thumb_tip.y < thumb_ip.y and all(f.y > middle_mcp.y for f in [index_tip, middle_tip, ring_tip, pinky_tip]):
        return "Thumbs Up"
    # Subtraction: Thumbs Down👎
    elif thumb_tip.y > thumb_ip.y and all(f.y > middle_mcp.y for f in [index_tip, middle_tip, ring_tip, pinky_tip]):
        return "Thumbs Down"
    # Multiplication: Open Hand🤚
    elif all(f.y < middle_mcp.y for f in [index_tip, middle_tip, ring_tip, pinky_tip]) and thumb_tip.y < thumb_ip.y:
        return "Open Hand"
    # Division: Rock Sign🤟
    elif index_tip.y < index_mcp.y and pinky_tip.y < middle_mcp.y and all(f.y > middle_mcp.y for f in [middle_tip, ring_tip]):
        return "Rock Sign"
    # Confirm input: Peace Sign✌
    elif (hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < 
          hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y and
          hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y < 
          hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y and
          hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y > 
          hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y and
          hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y > 
          hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y):
        return "Peace Sign"
    # Calculation: OK Sign (finger and thumb touching) 👌
    elif calculate_distance(thumb_tip, index_tip) < 0.05 and all(tip.y > middle_mcp.y for tip in [middle_tip, ring_tip, pinky_tip]):
        return "OK Sign"
    
    return "None"
def get_operator_from_gesture(gesture):
    if gesture == "Thumbs Up": 
        return "+"
    elif gesture == "Thumbs Down":
        return "-"
    elif gesture == "Open Hand":
        return "*"
    elif gesture == "Rock Sign":
        return "/"
    else:
        return None
# Initialize the camera
cap = cv2.VideoCapture(0)

state = "input1"
operand1 = ""
operator = ""
operand2 = ""
display_text = ""

# Variables to store pending input
pending_digit = None
pending_operator = None

# Variables for confirmation delay (for Peace Sign)
confirmation_delay = 1.0  # delay in seconds
peace_sign_start = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    current_gesture = None
    digit = None

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            gesture = get_gesture(hand_landmarks)
            current_gesture = gesture

            # If the gesture is not "Peace Sign", update the pending digit value
            if current_gesture != "Peace Sign":
                finger_count = count_fingers(hand_landmarks)
                digit = finger_count
                if digit is not None:
                    if pending_digit is None or pending_digit != digit:
                        pending_digit = digit
                        
            # If the current gesture is not Peace Sign, reset the confirmation timer
            if current_gesture != "Peace Sign":
                peace_sign_start = None

            # Handling calculator states
            if state == "input1":
                # If an operator is found and at least one character has already been entered
                op_candidate = get_operator_from_gesture(gesture)
                if op_candidate is not None and operand1 != "":
                    if pending_operator is None or pending_operator != op_candidate:
                        pending_operator = op_candidate

                # Confirmation of entry (if Peace Sign is recorded within the required time)
                if current_gesture == "Peace Sign":
                    if peace_sign_start is None:
                        peace_sign_start = time.time()
                    elif time.time() - peace_sign_start >= confirmation_delay:
                        if pending_operator is not None:
                            operator = pending_operator
                            pending_operator = None
                            state = "input2"
                        elif pending_digit is not None:
                            operand1 += str(pending_digit)
                            pending_digit = None
                        peace_sign_start = None  # reset timer after confirmation

            elif state == "input2":         
                # Update pending digit input for second operand if gesture not confirmed
                if current_gesture != "Peace Sign":
                    finger_count = count_fingers(hand_landmarks)
                    digit = finger_count
                    if digit is not None:
                        if pending_digit is None or pending_digit != digit:
                            pending_digit = digit
                            
                # Input confirmation for the second operand (with delay)
                if current_gesture == "Peace Sign":
                    if peace_sign_start is None:
                        peace_sign_start = time.time()
                    elif time.time() - peace_sign_start >= confirmation_delay:
                        if pending_digit is not None:
                            operand2 += str(pending_digit)
                            pending_digit = None
                        elif pending_operator is not None:
                            operator = pending_operator
                            pending_operator = None
                        peace_sign_start = None


                # Perform calculation: if "OK Sign" gesture is detected and second operand is entered
                if current_gesture == "OK Sign" and operand2 != "":
                    try:
                        result_value = eval(operand1 + operator + operand2)
                        display_text = operand1 + " " + operator + " " + operand2 + " = " + str(result_value)
                        operand1 = str(result_value)
                    except Exception as e:
                        display_text = "Error: " + str(e)
                    operand2 = ""
                    operator = ""
                    state = "input1"
                    pending_digit = None
                    pending_operator = None

            # Generating text for display
            display_text = operand1 + " " + operator + " " + operand2
            if pending_digit is not None:
                display_text += " |Number... " + str(pending_digit)
            if pending_operator is not None:
                display_text += " |Operator... " + pending_operator

    cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.imshow('Gesture Calculator', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt

